In [ ]:
# import all important stuff
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
import unittest, time, re, os, shutil, glob, urllib2

In [ ]:
# credentials
email=""
passwd=""

# folders
download_folder = "/folder/to/download/files/to"

In [ ]:
# start chrome in automated mode
driver = webdriver.Chrome()
driver.implicitly_wait(30)

# login
driver.get("https://www.cobocards.com/app/login/")
driver.find_element_by_name("email").click()
driver.find_element_by_name("email").clear()
driver.find_element_by_name("email").send_keys(email)
driver.find_element_by_name("password").clear()
driver.find_element_by_name("password").send_keys(passwd)
driver.find_element_by_name("password").send_keys(Keys.ENTER)

# create dictionary with key=folderid and value=name
folders = dict()

# loop over all folders
for folder in driver.find_elements_by_class_name('folder'):
    # get the link inside a folder
    linkobject = folder.find_element_by_xpath('.//a')
    # get url to folder
    url = linkobject.get_attribute("href")
    # remove trailing slash
    if url.endswith('/'):
        url = url[:-1]
    # remove rest of url (we only want folder id)
    key = url.split('/').pop()
    # get name of folder
    name = linkobject.text
    # save everything
    folders[key] = name
    print "Added new folder: " + name + " (" + key + ")"


In [ ]:
# now go to each folder and get cardsets
for folderid in folders:
    # navigate to correct url
    driver.get("https://www.cobocards.com/app/folder/" + folderid)
    # loop over all cardsets
    for cardset in driver.find_elements_by_class_name('cardsetbox'):
        # get correct cardset element
        cardset2 =  cardset.find_element_by_class_name('cardset')
        # click on drop down menu (down arraow)
        cardset2.find_element_by_xpath( './/img[@src="/app/img/arrow_dropdown_red.png"]').click()
        # click on export button
        dropdown = cardset2.find_element_by_class_name('dropdown')
        dropdown.find_element_by_xpath('.//a[@data-ga-evt="export-option-home"]').click()
        # wait until the "ok" button is clickable
        wait = WebDriverWait(driver, 3000)
        wait.until(EC.element_to_be_clickable((By.ID, 'm_ok')))
        # click "ok" button
        driver.find_element_by_id("m_ok").click()
        # wait until export dialog has vanished
        wait = WebDriverWait(driver, 300)
        wait.until(EC.invisibility_of_element_located((By.ID, 'm_div')))
    
    # create a new folder with that name
    target_folder = download_folder + "/Exports" + "/" + folders[folderid]
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)
        
    # move all downloaded packages to this folder
    downloads = download_folder + "/CoboCards*.zip"
    for download in glob.glob(downloads):
        shutil.move(download, target_folder)
